# AWS Lambda - Twilio


## Setup



If you have chosen to use an rc file, source it as ```source <your-aws-credentials-rc-file>```, e.g.

In [ ]:
. ~/.aws/credentials.rc

# Interfacing with Twilio

For this functionality you will need to create a free account with Twilio and then validate the telephone numbers to which you wish to send SMS

You then need to export the following variables in your environment:

```
export TWILIO_ACCOUNT_SID='your-account-sid
export TWILIO_AUTH_TOKEN='your-auth-token
export TWILIO_FROM_NUMBER='twilio-provided-number'
export TWILIO_TO_NUMBER='the-number-to-send-SMS'
```

It is best to place them in a sourceable file such as ~/.twilio.rc

In [1]:
cd /home/user1/src/git/ServerlessLabs/ServerlessWorkshop/AWS-S3-Lambda/

chalice new-project twilio-sms

In [2]:
cd twilio-sms

cat > app.py <<EOF

from os import environ as env

# 3rd party imports
from chalice import Chalice, Response
from twilio.rest import Client
from twilio.base.exceptions import TwilioRestException

# Twilio Config
ACCOUNT_SID = env.get('TWILIO_ACCOUNT_SID')
AUTH_TOKEN = env.get('TWILIO_AUTH_TOKEN')
FROM_NUMBER = env.get('TWILIO_FROM_NUMBER')
TO_NUMBER = env.get('TWILIO_TO_NUMBER')

app = Chalice(app_name='sms-shooter')

# Create a Twilio client using account_sid and auth token
tw_client = Client(ACCOUNT_SID, AUTH_TOKEN)

@app.route('/service/sms/send', methods=['POST'])
def send_sms():
    request_body = app.current_request.json_body
    if request_body:
        custom_headers={'X-myheader': 'cool', 'X-From': "'"+FROM_NUMBER+"'", 'X-To': "'"+TO_NUMBER+"'", 'Content-Type': 'application/json'}
        try:
            msg = tw_client.messages.create(
                from_=FROM_NUMBER,
                body=request_body['msg'],
                to=TO_NUMBER)

            if msg.sid:
                return Response(status_code=201,
                                headers=custom_headers,
                                body={'status': 'success',
                                      'data': msg.sid,
                                      'message': 'SMS <{}> successfully sent to {}'.format(request_body['msg'], TO_NUMBER)})
            else:
                return Response(status_code=200,
                                headers=custom_headers,
                                body={'status': 'failure',
                                      'message': 'Please try again!!!'})
        except TwilioRestException as exc:
            return Response(status_code=400,
                                headers=custom_headers,
                            body={'status': 'failure',
                                  'message': exc.msg})
                                  

EOF

cat > requirements.txt <<EOF
boto3==1.3.1
twilio==6.22.0
EOF

### Sending SMS from local server

Stop the local server ```ctrl-C```

Source ~/.twilio.rc
```.  ~/.twilio.rc```

Restart local server
```chalice local```


In [3]:
. ~/.twilio.rc
#http POST :8000/service/sms/send msg="MY MESSAGE" --debug
http POST :8000/service/sms/send msg="MY MESSAGE at $(date) from ${USER}@$(hostname)"

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
HTTP/1.1 201 Created
Content-Length: 187
Content-Type: application/json
Date: Sat, 26 Jan 2019 06:48:04 GMT
Server: BaseHTTP/0.6 Python/3.6.7
X-From: '+33644601324'
X-To: '+33652891534'
X-myheader: cool

{
    "data": "SM7db3a082946e411cb95eec2e44cc7149",
    "message": "SMS <MY MESSAGE at Sat Jan 26 06:48:04 UTC 2019 from user1@ip-172-31-21-116> successfully sent to +33652891534",
    "status": "success"
}



### Sending SMS to Twilio from deployed service

**INCOMPLETE**: Need to safely transfer credentials into version deployed onto AWS

In [ ]:
chalice deploy; chalice url

In [ ]:
## BUT HOW TO PASS IN TWILIO SECRETS ?? !!

http POST https://jlmq3ygdu1.execute-api.us-west-1.amazonaws.com/api/service/sms/send msg="MY MESSAGE at $(date) from ${USER}@$(hostname)"

# More Reading


# Cleanup

